In [22]:
transcripts=[]
transcripts.append(['ENST00000456328.2',0])
transcripts.append(('ENST00000450305.2',0))
transcripts[0][1]=transcripts[0][1]+1
print(transcripts[0][1])

1


In [4]:
import pandas as pd
reference_file_path_mut="/data01/private/projects/splicing_cll/results/proteomics/analysis.20230713/mutated_cll_sf3b1_proteomics_reference.txt"
reference_file_path_unmut="/data01/private/projects/splicing_cll/results/proteomics/analysis.20230713/unmutated_cll_sf3b1_proteomics_reference.txt"
reference_file_mut=pd.read_csv(reference_file_path_mut,sep='\t',header=None,names=["chr","start","end","strand","ENST","ENSG"])
reference_file_unmut=pd.read_csv(reference_file_path_unmut,sep='\t',header=None,names=["chr","start","end","strand","ENST","ENSG"])
reference_file=pd.concat([reference_file_mut,reference_file_unmut])
reference_file.drop_duplicates(keep='first',inplace=True)
print(len(reference_file["ENST"].unique()))

463


In [17]:
import pysam

bam_path = "/data01/private/data/cllmap/data/rna/bams/DKFZ-CLL65.Aligned.sortedByCoord.out.bam"  # Replace with your BAM file path

# Open the BAM file
with pysam.AlignmentFile(bam_path, "rb") as bam_file:
    # for line in bam_file.head(10):
    #     print(line)
   
    # get header information
    # header = bam_file.header
    # print(header)
    # libraries = set()

    # # get transcriptome information
    # transcriptome = bam_file.references
    # print(transcriptome)
    print(bam_file.header.get("RG"))
    # # Iterate through the read groups in the header
    # for rg_dict in bam_file.header.get("RG", []):
    #     print(rg_dict)

    # # 
    # # Print the unique library IDs
    # for library in libraries:
    #     print(library)

[{'ID': 'rg1', 'SM': 'sm1'}]


In [8]:
transcripts=[]
with open("/home/ls/rachelcw/projects/salmon/transcriptome.fa", "r") as f:
    for line in f:
        line=line.strip()
        if line.startswith(">"):
            transcript=line.split("|")[1]
            transcripts.append(transcript)


# Read in the data


In [11]:
print(len(transcripts),len(set(transcripts)))

416 379
